In [3]:
# import anything we need in this notebook
using Binsum

# Chapter 2: Introduction to Approximation

In this notebook we collect the computational examples associated with Chapter 2 of *Multiscale Simulation*. 

**Remark:** These codes use some elementary functional style constructs. 
These may initially require some getting used to, but in the long run they
make for code that is brief, readable and less likely to contain bugs.
In particular, Julia encourages to avoid `if` statements and instead use
its dispatch mechanism. This is a paradigm that makes for simple and 
elegant code.

## Model

The homogeneous lattice model is
$$
    E(u) = \sum_{\ell \in \mathbb{Z}^2} \sum_{i = 1}^2 \phi\big( D_i u(\ell) \big).
$$
We will use $\phi(r) = \sin^2( \pi r )$ throughout, which models an 
anti-plane nearest-neighbour pair interaction. 

We can introduce a point defect by introducing additional lattice sites and
associated bonds. A screw dislocation is introduced by adding a *pre-strain* 
to the bonds.  In order to implement only one model, we will describe both
of these within the same framework.

Let $\Lambda$ be a point defect reference configuration and $B$ an
associated set of nearest-neighbour bonds. Specifically we consider two
cases:

* *Homogeneous lattice:* $\Lambda^{\rm hom} = \mathbb{Z}^2$ and 
    $B^{\rm hom} = \{ (i,j) \in \Lambda^2 : |i-j| = 1 \}$.
* *Interstitial-type defect:* $\Lambda^{\rm int} = \mathbb{Z}^2 \cup \{\xi\}$, where $\xi = (4/7, 1/3)$ is the defect site
and $B^{\rm int} = \{(i,j) \in \Lambda^2 : |i-j| \leq 1 \}$.

The pre-strain is a map ${\sf e} : B \to \mathbb{R}$. The energy is now given by
$$
  E(u) = \sum_{b \in B} \phi( {\sf e}_b + Du_b ) - \phi( {\sf e}_b ).
$$

We define the two benchmark problems through specifying $\Lambda, B, {\sf e}$:

* **Point defect case** 
    - $\Lambda = \Lambda^{\rm int}$
    - $B = B^{\rm int}$
    - ${\sf e}_{(i,j)} = 1 - |i-j|$
* **Screw dislocation case** 
    - $\Lambda = \Lambda^{\rm hom}$
    - $B = B^{\rm hom}$
    - ${\sf e}_{(i,j)} = (\frac{1}{2\pi} \arg(i-\xi) - \frac{1}{2\pi}\arg(j-\xi))~{\rm mod}~(-1/2, 1/2]$

More details about these choices can be found in Chapters 3 and 4 of the book.


## ATM Approximation

We first consider the purely atomistic approximation where only the
admissible displacements are restricted. For $R \in \mathbb{N}$ the
computational domain is given by
$$
   \Omega_R = \Lambda \cap [-R+1, R]^2
$$
with Dirichlet conditions (clamped) on the boundary sites. 



In [58]:
# the defect core position
const ξ = [3/7; 1/3]

# [1] Model type : to collect all the information
#     the construction is produced below

type Model
    R   # domain parameter
    Ω   # lattice coordinates
    B   # bonds
    e   # pre-strains
    Ifree   # free dofs
    defect  # type of defect
end

"number of lattice sites in the computational domain"
nsites(m::Model) = length(m.Ω[1])
"number of dofs"
ndofs(m::Model) = length(m.Ifree)

# [2] Domain

"""computational domain; returns (x, y) coordinate 
vectors of lattice points"""
domain(R::Int, ::Val{:none}) = domain_hom(collect(-R+1:R))
domain(R::Int, ::Val{:screw}) = domain(R, Val{:none}())
domain(R::Int, ::Val{:int}) = domain_int(domain(R, Val{:none}()))
domain_hom(t::Vector) = ( (t * ones(t)')[:], (ones(t) * t')[:] )
domain_int(Ω::Tuple) = ([Ω[1]; ξ[1]], [Ω[2]; ξ[2]])


# [3] bonds

"bonds within a computational domain"
bonds(R::Int, ::Val{:none}) = bonds_hom(R, (2*R, 2*R))
bonds(R::Int, ::Val{:screw}) = bonds(R::Int, Val{:none}())
bonds(R::Int, ::Val{:int}) = bonds_int(R, bonds(R, Val{:none}()), defbonds(R))
bonds_hom(R::Int, sz) = 
        ( [ [sub2ind(sz, i, j) for i in 1:2*R-1, j in 1:2*R][:];
            [sub2ind(sz, i, j) for i in 1:2*R, j in 1:2*R-1][:] ],
          [ [sub2ind(sz, i+1, j) for i in 1:2*R-1, j in 1:2*R][:];
            [sub2ind(sz, i, j+1) for i in 1:2*R, j in 1:2*R-1][:] ] )
defneigs(R) = Int[ (R-1)*(2*R)+R-1; (R-1)*(2*R)+R; R*(2*R)+R-1; R*(2*R)+R ]
defbonds(R) = (defneigs(R), (4*R^2+1) * ones(Int, 4))
bonds_int(R::Int, Bhom, Bdef) = ([Bhom[1]; Bdef[1]], [Bhom[2]; Bdef[2]])


# [4] prestrain is a function of lattice positions

fdiff(v, B) = v[B[2]] - v[B[1]]
prestrain(Ω, B, ::Val{:none}) = zeros(length(B[1]))
prestrain(Ω, B, ::Val{:int}) = 
            1.0 - sqrt( fdiff(Ω[1], B).^2 + fdiff(Ω[2], B).^2 )
prestrain(Ω, B, ::Val{:screw}) = 
            prestrain_screw(Ω[1][B[1]], Ω[1][B[2]], Ω[2][B[1]], Ω[2][B[2]])
arg(x, y) = imag(log(x + im * y))
prestrain_screw(x1, x2, y1, y2) = 
            mod( (arg(x2, y2) - arg(x1, y1)) / (2*π) + 0.5, 1.0 ) - 0.5


# [5] free indices

Ifree(R, ::Val{:none}) = 
            (reshape(collect(1:(2*R)^2), (2*R,2*R))[2:end-1,2:end-1])[:]
Ifree(R, ::Val{:screw}) = Ifree(R, Val{:none}())
Ifree(R, ::Val{:int}) = [Ifree(R, Val{:none}()); (2*R)^2 + 1]


# [6] constructor for the model type

function Model(; R=5, defect=:none ) 
    valdef = Val{defect}()
    Ω = domain(R, valdef)
    B = bonds(R, valdef)
    return Model(R, Ω, B, prestrain(Ω, B, valdef), Ifree(R, valdef), defect)
end


# [7] Energy as a function of displacement

"NN pair potential (we use the same potential for all experiments)" 
ϕ(r) = sin(r*π).^2
"first derivative of ϕ"
Dϕ(r) = π * sin(2*r*π)
"second derivative of ϕ"
D²ϕ(r) = 2*π^2 * cos(2*r*π)

"potential energy (difference), as function of displacement"
energy_disp(m, U) = sum_kbn( ϕ(m.e + fdiff(U[:], m.B)) - ϕ(m.e) )

"gradient of potential energy"
grad_disp(m, U) = _grad_(m.B, Dϕ(m.e + fdiff(U[:], m.B)))
_grad_(B, dϕ) = binsum([dϕ; -dϕ], [B[2]; B[1]])

"hessian as function of displacement"
hess_disp(m, U) = _hess_(m.B, D²ϕ(m.e + fdiff(U[:], m.B)))
_hess_(B, hϕ) = sparse( [B[1]; B[2]; B[1]; B[2]], 
                        [B[1]; B[2]; B[2]; B[1]],  
                        [  hϕ;   hϕ;  -hϕ; -hϕ] )

# [8] displacement ↔ dof conversion

"create a dof vector from a displacement vector"
disp2dof(m, U) = U[m.Ifree]

"create a full displacement vector from a dof vector"
function dof2disp(m, W)
    U = zeros(nsites(m))
    U[m.Ifree] = W[:]
    return U
end

"energy as function of dofs"
energy(m, W) = energy_disp(m, dof2disp(m, W))
"gradient as function of dofs"
grad(m, W) = grad_disp(m, dof2disp(m, W))[m.Ifree]
"hessian as function of dofs"
hess(m, W) = hess_disp(m, dof2disp(m, W))[m.Ifree, m.Ifree]


# [9] Basic Newton solver 
#     (this will be sufficient for our purposes here)

function solve(m; x = zeros(ndofs(m)), show = false)
    show && @printf("------Newton Iteration------\n")
    nit = 0
    for nit = 0:9
        ∇E = grad(m, x)
        show && @printf("%d : %4.2e \n", nit, norm(∇E, Inf))
        norm(∇E, Inf) < 1e-8 && break
        x = x - hess(m, x) \ ∇E
    end
    nit == 9 && warning("the Newton iteration did not converge")
    show && @printf("----------------------------\n")
    return x
end
;

### Testing the implementation

We perform some basic tests to make sure the implementation is correct. 
The basic philosophy of these tests is that, if the energy, gradient and 
hessian are all consistent then the implementation is likely correct. 
Hense we perform a finite difference test. We will test all partial derivatives
for decreasing steps $h$.

Note that
$f'(x) = \frac{f(x+h) - f(x)}{h} + O(h)$, but when we take floating-point errors 
into account, then we obtain
$$
    f'(x) = \frac{f(x+h) - f(x)}{h} + O\left( h + \frac{\epsilon}{h} \right),
$$
where $\epsilon$ denotes floating point accuracy. Hence, as $h$ decreases we first see an improvement but eventually a deterioration of the test agreement.

In [60]:
"perform finite difference test for an energy functional, gradient and hessian"
function fdtest(U, energy_, grad_, hess_)
    E = energy_(U)
    ∇E = grad_(U)
    ∇²E = full(hess_(U))
    @printf("    h  | ∇E-error   ∇²E-error  \n")
    @printf("-------|----------------------- \n")
    for p = 2:12
        h = 0.1^p
        ∇E_h = zeros(∇E)
        ∇²E_h = zeros(∇²E)
        for n = 1:length(U)
            U[n] += h
            ∇E_h[n] = (energy_(U) - E) / h
            ∇²E_h[:,n] = (grad_(U) - ∇E) / h
            U[n] -= h
        end
        @printf(" %1.0e | %4.2e   %4.2e  \n", 
                h, norm(∇E - ∇E_h, Inf), vecnorm(∇²E - ∇²E_h, Inf))
    end
    return nothing
end

# run the FD test : change `defect` to :none, :int, :screw
# to test the three implementations
m = Model(R = 5, defect = :screw)
# println("Test Energy as function of displacement")
# fdtest(0.1 * rand(nsites(m)), x->energy_disp(m,x), x->grad_disp(m,x), x->hess_disp(m,x))
println("Test Energy as function of dofs")
fdtest(0.1 * rand(ndofs(m)), x->energy(m,x), x->grad(m,x), x->hess(m,x))


Test Energy as function of dofs
    h  | ∇E-error   ∇²E-error  
-------|----------------------- 
 1e-02 | 3.92e-01   1.03e+00  
 1e-03 | 3.92e-02   9.85e-02  
 1e-04 | 3.92e-03   9.81e-03  
 1e-05 | 3.92e-04   9.81e-04  
 1e-06 | 3.92e-05   9.81e-05  
 1e-07 | 3.92e-06   9.80e-06  
 1e-08 | 3.67e-07   1.09e-06  
 1e-09 | 7.88e-07   1.78e-06  
 1e-10 | 6.12e-06   2.17e-05  
 1e-11 | 7.72e-05   1.88e-04  
 1e-12 | 6.47e-04   1.88e-03  


Our second test concerns the Newton scheme. If it is correctly 
implemented and if gradient and hessian are consistent (we just
tested this), then it should converge in 4-5 iterations at most.

In [67]:
# change `defect` to {:none, :int, :screw} for the three model problems
m = Model(R = 100, defect=:none)
solve( m; show=true, x = 0.1 * rand(ndofs(m)) );

------Newton Iteration------
0 : 6.78e+00 
1 : 8.96e-01 
2 : 1.55e-03 
3 : 7.97e-12 
----------------------------


### Visualise the Solutions

### Testing the rate of decay

### Testing the Convergence Rate for the `ATM` Model